In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import distances
import re
import matplotlib.pyplot as plt
#import nglview as nv
import numpy as np
import multiprocessing
import pickle
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import pickle

In [25]:

 
 
# Create a dictionary of systems in a folder given a path and a tag
def createSystemsDict(path, tag):
   systems = {}
   for file in os.listdir(path):
       # If the file is a directory and contains the tag
       if os.path.isdir(os.path.join(path, file)) and tag in file:
           systems[file] = os.path.join(path, file)
   return systems
 
 
 
# Given a system name, generate a dictionary of mda.Universes
def createUniverseDict(systemsDict,systemName):
   trajFormat = '.dcd'
   lengthMD = '100ns'
   systemPath = systemsDict[systemName]
   # Print creating universe for system
   print('Creating universe for system: ' + systemName)
   universesMDByReplicaDict = {}
   universeDictAllReplicas = {}
   numberOfReplicas = len([f for f
                   in os.listdir(os.path.join(systemPath, 'processed_trajectories'))
                       if
                           re.match(r'r*', f)
                       and
                           f.endswith(trajFormat)
                       and
                           lengthMD in f
               ]
           )
 
   # Print the number of replicas by counting the number of files that have r* in their name using regex
   print('Number of replicas: ', numberOfReplicas)
   # For each replica inside the folder processed_trajectories
   for replica in range(1, numberOfReplicas + 1):
       # Create a universe with the replica number and the system name
       # Use regex to match the replica: md_dry_100ns_r*.nc
       # Append the universe to a list of universes
       universeDictAllReplicas[replica] = mda.Universe(os.path.join(systemPath, 'processed_trajectories','snapshot.pdb'),
                               os.path.join(systemPath, 'processed_trajectories', 'md_dry_100ns_r' + str(replica) + '.dcd'))
   # Append the dictionary of universes to a dictionary with the systemName as key
   # and the dictionary of universes as values
   universesMDByReplicaDict[systemName] = universeDictAllReplicas
   return universeDictAllReplicas
 
# Given a universe, two residue numbers and two atom lists, create a MDanalysis selection
 
def createSelection(replicaUniverse, residueNumber1, residueNumber2, atomList1, atomList2):
   # Create a empty dictionary
   selectionDict = {}
   # Create a selection for each atom in the atom list
   for atom in atomList1:
       selectionDict[atom] = replicaUniverse.select_atoms('resid ' + str(residueNumber1) + ' and name ' + atom)
   for atom in atomList2:
       selectionDict[atom] = replicaUniverse.select_atoms('resid ' + str(residueNumber2) + ' and name ' + atom)
   return selectionDict
 
tag  = '130@OG-EMT@'
# Given a selection dictionary, calculate the distances between the first atom in the list and the rest of the atoms in the list
def calculateDistances(selectionDict):
   distancesDict = {}
   for atom in selectionDict:
       # if the atom is not the first atom in the list
       if atom != list(selectionDict.keys())[0]:
           # Use a list comprehension to calculate the distance between the first atom in the list and second atom in the list
           # and append the distance to a numpy array
           # iterate over the frames in the universe
           # save only the third element of the numpy array returned by the distance function
           distancesDict[f'{tag}{atom}'] = np.array([distances.dist(selectionDict[atom],
                                                       selectionDict[list(selectionDict.keys())[0]],
                                                       offset=0)[2] for ts in selectionDict[atom].universe.trajectory])
   return distancesDict
 
# Plot the distances in the distances dictionary
# Use a len(distancesDict) by 1 grid
def plotDistances(distancesDict, systemName,replica):
 
   # Create a figure
   fig, axs = plt.subplots(len(distancesDict), 1, figsize=(5, 50))
   # Set a qualitative color palette
   colors = plt.cm.tab10(np.linspace(0, 1, len(distancesDict)))
 
   # For each distance in the distances dictionary
   for i, distance in enumerate(distancesDict):
       # Plot the distance
       axs[i].plot(distancesDict[distance])
       # Set the title of the plot
       axs[i].set_title(distance.replace('130', '131'))
       # Set the y axis range to be between 0 and the maximum value of the distance plus 1
       axs[i].set_ylim(0, 20)
       # Add a span in the y axis from 2 to 4
       axs[i].axhspan(2.5, 4.5, zorder=0, alpha=0.2, color='orange', label='H-bond')
       # Add the span to the legend
       axs[i].legend()
       # Make the line color the color in the color palette
       axs[i].lines[0].set_color(colors[i])
       # Make the title of the plot bold
       axs[i].title.set_fontweight('bold')
       # Make the plot look nice
       # Use the white grid style
       plt.style.use('seaborn-whitegrid')
       # Increase the font size
       plt.rcParams.update({'font.size': 15})
       # Increase the line width
       plt.rcParams['lines.linewidth'] = 4
       # Increase space between subplots
       plt.subplots_adjust(hspace=0.3)
       # Set the x axis label
       axs[i].set_xlabel('Frame')
       # Set the y axis label
       axs[i].set_ylabel('Distance (Å)')
       # Set the axis labels to be bold
       axs[i].xaxis.label.set_fontweight('bold')
       axs[i].yaxis.label.set_fontweight('bold')
       # Set the axis ticks to be bold
       axs[i].xaxis.get_offset_text().set_fontweight('bold')
       axs[i].yaxis.get_offset_text().set_fontweight('bold')
       # Convert the x axis from 20000 frames to 100 ns
       #axs[i].set_xticks(np.arange(0, 20000, 2000))
 
 
      
   # Show the plot
   #plt.show()
   # Save the plot in the figures folder using the system name and the residue number
   fig.savefig(os.path.join(systemsFolder +'/figures/' + systemName + '_' + replica + '_distances'+ '.png'),dpi=300, bbox_inches='tight')
   # Save a pdf version of the plot
 
def plotDistancesV2(
   distancesDict: str,
   systemName: str
):
   # Create a list of dataframes for all replicas
   dfList = []
   # For each replica in the distances dictionary
   
  
  
 
 
def calculateDistancesParallelFunction(systemName: str ,
                                       replicaNumber: str,
                                       serineResidueNumber: int,
                                       petResidueNumber: int,
                                       serine_atom_list: list,
                                       pet_atoms_list: list,
                                       systemsFolder: str,
                                       iguresFolder: str):
   """AI is creating summary for calculateDistancesParallelFunction
 
   Args:
       systemName ([type]): [description]
       replicaNumber ([type]): [description]
       serineResidueNumber ([type]): [description]
       petResidueNumber ([type]): [description]
       serine_atom_list ([type]): [description]
       pet_atoms_list ([type]): [description]
       systemsFolder ([type]): [description]
       figuresFolder ([type]): [description]
   """
   # Create a dictionary of systems
   systemsDict = createSystemsDict(systemsFolder, '298')
   # Create a dictionary of universes
   universeDictByReplica = createUniverseDict(systemsDict, systemName)
   # Create a selection dictionary
   selectionDict = createSelection(universeDictByReplica[replicaNumber], serineResidueNumber, petResidueNumber, serine_atom_list, pet_atoms_list)
   # Calculate the distances
   distancesDict = calculateDistances(selectionDict)
   # Plot the distances
   plotDistances(distancesDict, systemName, str(replicaNumber))
   # Create the pickle folder if it does not exist
   if not os.path.exists(os.path.join(systemsFolder, pickleFolder)):
         os.makedirs(os.path.join(systemsFolder, pickleFolder))
   # Save the distances in a pickle file in the
   with open(os.path.join(systemsFolder, pickleFolder, systemName + '_' + str(replicaNumber) + '_distances.pickle'), 'wb') as handle:
       pickle.dump(distancesDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
 
# A function to load the distances from a pickle file and return a dictionary of distances
def loadDistances(systemsFolder,systemName, replicaNumber):
   allDistancesDict = {}
   with open(os.path.join(systemsFolder, pickleFolder, systemName + '_' + str(replicaNumber) + '_distances.pickle'), 'rb') as handle:
       distancesDict = pickle.load(handle)
       # Append the distances dictionary to a dictionary with the system name as key
       allDistancesDict[systemName] = distancesDict
   return allDistancesDict
  
import imp
from operator import index
import pandas as pd
def createDataframeFromDistancesDict(
   distancesDict: dict
):
   # Create a dataframe to store the distances for the dictionary in the distances dictionary
   distancesDataFrame = pd.DataFrame()
# For each system in the distances dictionary
   for system in distancesDict:
   # For each key in the distances dictionary append it as a new column in the dictionary
       for key in distancesDict[system]:
       # Each element in the value is a numpy array, so we need to convert it to a float
           distancesDataFrame[key] = distancesDict[system][key].tolist()
       # Convert the one element list to a float
           distancesDataFrame[key] = distancesDataFrame[key].apply(lambda x: x[0])
        
   return distancesDataFrame

import os
import openpyxl
import zipfile

def load_workbook(filename=None):
    if filename and os.path.exists(filename) and zipfile.is_zipfile(filename):
        return openpyxl.load_workbook(filename)
    else:
        return openpyxl.Workbook()

import plotly.graph_objects as go

def plotDataFrameDistances(df, systemName, replicaNumber):
   # Plot the distances with the rolling average and standard deviation for all columns that have the word 'Sampled_Avg' in them
    fig = go.Figure()
    # Create a range of colors
    colors = ['rgb(31, 119, 180)',
         'rgb(255, 127, 14)',
         'rgb(44, 160, 44)',
         'rgb(214, 39, 40)',
         'rgb(148, 103, 189)',
         'rgb(140, 86, 75)',
         'rgb(227, 119, 194)',
         'rgb(127, 127, 127)',
         'rgb(188, 189, 34)',
         'rgb(23, 190, 207)']
    colors_transparent = ['rgba(31, 119, 180, 0.2)',
           'rgba(255, 127, 14, 0.2)',
           'rgba(44, 160, 44, 0.2)',
           'rgba(214, 39, 40, 0.2)',
           'rgba(148, 103, 189, 0.2)',
           'rgba(140, 86, 75, 0.2)',
           'rgba(227, 119, 194, 0.2)',
           'rgba(127, 127, 127, 0.2)',
           'rgba(188, 189, 34, 0.2)',
           'rgba(23, 190, 207, 0.2)']
    for column in df:
        # If the column has the word 'Sampled_Avg' in it or Std_Dev in it
       if 'Sampled_Avg' in column or 'Std_Dev' in column:
        # If the column has the word 'Sampled_Avg' in it
           if 'Sampled_Avg' in column:
                # If the replica number is 4
                # legend = True
                # Else set the legend to false
                if replicaNumber == 4:
                        # Add the go.Scatter object to the figure for the rolling average of the particular distance
                    fig.add_trace(go.Scatter
                    (
                        name=column.split('_')[0].split('-')[1],
                        x=df['Time(ns)'],
                        y=df[column],
                        mode='lines',
                        line=dict(color=colors[0]),
                        showlegend=True,
                        # Make the line thicker
                        line_width=5
                    )
                )
                else:
                    fig.add_trace(go.Scatter
                    (
                        name=column.split('_')[0].split('-')[1],
                        x=df['Time(ns)'],
                        y=df[column],
                        mode='lines',
                        line=dict(color=colors[0]),
                        showlegend=False,
                        line_width=5
                    )
                )
                
           # Remove the first color from the list
                colors.pop(0)
                fig.add_trace(go.Scatter
                (
                   name=f'{column}_Lower_Bound',
                   x=df['Time(ns)'],
                   y=df[str(column)[:-12] + '_Std_Dev']+df[column],
                   mode='lines',
                   marker=dict(color="#444"),
                   line=dict(width=0),
                   showlegend=False
                )
            )
          
                fig.add_trace(go.Scatter
               (
                   # Name the trace the same as the column + lower bound
                   name=f'{column}_Lower_Bound',
                   # Set the x axis to the time
                   x=df['Time(ns)'],
                   # Set the y axis to the rolling average - the standard deviation
                   y=df[column]-df[str(column)[:-12] + '_Std_Dev'],
                   # Set the mode to lines
                   mode='lines',
                   # Set the marker to be transparent
                   marker=dict(color="#444"),
                   # Set the line width to 0
                   line=dict(width=0),
                   # Set the fill color to be transparent
                   fillcolor=colors_transparent[0],
                   # Set the fill to be tonexty
                   fill='tonexty',
                   # Set the showlegend to be false
                   showlegend=False
               )
           )
                colors_transparent.pop(0)
                # Add a rectangle to the figure from y0=2.5 to y1=4.5
                # Add shapes
                fig.add_hrect(
               y0=2.5, y1=5,
               fillcolor="LightSalmon", opacity=0.03,
               layer="below", line_width=0,
           )
                                  
    # If the replica number is 1
    if replicaNumber == 1:
        # Set the y axis title to be 'Distance (angstroms)'
        fig.update_layout(yaxis_title='<b><br>Distance to S131@OG (Å)</b>')
    # Set the x axis title to be 'Time (ns)'
    fig.update_layout(xaxis_title='Time (ns)')
    # Set the theme to be 'plotly_white'   
    fig.update_layout(template='plotly_white')   
    # If the system name is contains 'L210A'
    if 'L210A' in systemName:        
        # Set the title to be the system name and the replica number replace the _ with ' '
        if replicaNumber == 1:

            fig.update_layout(
                title={
                    # Add the system name and the replica number use <br> to break the line
                    'text': f'<b>{systemName.replace("_", " ").replace("2PET","EMT").replace("298","298K")}</b><br>Replica {replicaNumber}',
                    'y':0.95,
                    'x':0.65,
                    'xanchor': 'center',
                    'yanchor': 'top'})

            
        else:
            fig.update_layout(
                title={
                    # Add the system name and the replica number use <br> to break the line
                    'text': f'<br>Replica {replicaNumber}',
                    'y':0.95,
                    'x':0.45,
                    'xanchor': 'center',
                    'yanchor': 'top'})
    else:
        # Set the title to be the system name and the replica number replace the _ with ' '
        if replicaNumber == 1:

            fig.update_layout(
                title={
                    # Add the system name and the replica number use <br> to break the line
                    'text': f'<b>{systemName.replace("_", " ").replace("2PET","EMT").replace("298","298K")}</b><br>',
                    'y':0.95,
                    'x':0.65,
                    'xanchor': 'center',
                    'yanchor': 'top'})

        
    # Make the plot square
    fig.update_layout(height=800, width=1200)
    # If the replica number is 1 make the width sligtly bigger to account by the y axis legend
    if replicaNumber == 1:
        fig.update_layout(height=800, width=1250)
    # If the replica number is 4 make the width bigger to account by the legends
    if replicaNumber == 4:
        fig.update_layout(height=800, width=1250)

    
    # Center the title
    fig.update_layout(title_x=0.5)
    # Increase the font size of everything
    fig.update_layout(font=dict(size=40))
    #fig.show()
    # Set the y axis range to be from 0 to 20
    fig.update_yaxes(range=[0, 20])
    # Set the layout tight
    fig.update_layout(margin=dict(l=0, r=10, t=100, b=0))
    # Make the axis lines thicker
    fig.update_xaxes(showline=True, linewidth=5, linecolor='black', mirror=True)
    # Make the axis lines thicker
    fig.update_yaxes(showline=True, linewidth=5, linecolor='black', mirror=True)
    # Move the title up so it doesn't overlap with the axis
    fig.update_layout(title_y=0.9)
    # Set the x axis label to be Time (ns) in bold
    fig.update_xaxes(title_text='<b>Time (ns)</b>')
    # Add black outside ticks to the x axis gfand y axis
    fig.update_xaxes(showticklabels=True, ticks='outside', tickwidth=5, tickcolor='black', ticklen=15)
    fig.update_yaxes(showticklabels=True, ticks='outside', tickwidth=5, tickcolor='black', ticklen=15)

    # Set the x axis length static independant of the legend
    fig.update_xaxes(automargin=True)
    # Save the figure as a pdf in the figures folder
    fig.write_image(f'{systemsFolder}/figures/{systemName}_{replicaNumber}_distances.png', scale=2)
    # Print that the figure has been saved
    print(f'{systemsFolder}/figures/{systemName}_{replicaNumber}_distances.png has been saved')
    # Create an excel writer
    # if the replica number is 1
    # import ExcelWriter from pandas
    from pandas import ExcelWriter
    # Create an excel writer
    writer = pd.ExcelWriter(f'{systemsFolder}/figures/{systemName}_Replica{replicaNumber}_distances.xlsx', engine='xlsxwriter')
    # Convert the dataframe to an xlsx
    df.to_excel(writer, sheet_name='Sheet1')
    # Close the excel writer
    writer.save()

    # Return the figure
    return fig
def loadAndPlotReplicaSystemFromPickle(total_time: int, 
                                       systemName: str,
                                       dfList: list,
                                       replicaNumber: int
                                       ):
    # Load the distances from the pickle file
    distancesDict = loadDistances(systemsFolder, systemName, replicaNumber)
       # Create a dataframe from the distances dictionary
    distancesDataFrame = createDataframeFromDistancesDict(distancesDict)
       # For each column in the dataframe calculate the Rolling Average and standard deviation
       # and append it to the dataframe as '130@OG-EMT@O9_RollingAVG' and '130@OG-EMT@O9_std' for example
    for column in distancesDataFrame:
        distancesDataFrame[column + '_Sampled_Avg'] = distancesDataFrame[column].rolling(200).mean()
        distancesDataFrame[column + '_Std_Dev'] = distancesDataFrame[column].rolling(200).std()
    distancesDataFrame['Time(ns)'] = np.arange(0, total_time, total_time/len(distancesDataFrame))
       # Append the dataframe to the list
    dfList.append(distancesDataFrame)
       # Create a figure for the system and replica
    plotDataFrameDistances(distancesDataFrame, systemName, replicaNumber)





## Dowload example data

In [1]:
!pip install -q gdown

In [4]:
!pwd

/home/iwe30/Github/md-analysis-utils/protein-ligand


In [5]:
# Download the files from the google drive
# https://drive.google.com/file/d/1SChRa8U5UPzLD9hTJLBhu-KouElG2w46/view?usp=sharing
# https://drive.google.com/file/d/1Jz5N3jaumcSxLKEUqvO7Oruj_pXt0xhR/view?usp=sharing
# https://drive.google.com/file/d/1qodaf391nRNUGM4WqR1TaxwMK_q_f9vJ/view?usp=sharing
# https://drive.google.com/file/d/1OtEHAkJ-OhrPmtTor0Pfr3d_YVNvLIWX/view?usp=sharing

# Download the files from the google drive inside the folder 
# "protein-ligand/example-data/WT_298_2PET_RESP/processed_trajectories"
# md_dry_100ns_r1.dcd
# md_dry_100ns_r2.dcd
# md_dry_100ns_r3.dcd
# md_dry_100ns_r4.dcd

!gdown --id 1SChRa8U5UPzLD9hTJLBhu-KouElG2w46 -O example-data/WT_298_2PET_RESP/processed_trajectories/md_dry_100ns_r1.dcd
!gdown --id 1Jz5N3jaumcSxLKEUqvO7Oruj_pXt0xhR -O example-data/WT_298_2PET_RESP/processed_trajectories/md_dry_100ns_r2.dcd
!gdown --id 1qodaf391nRNUGM4WqR1TaxwMK_q_f9vJ -O example-data/WT_298_2PET_RESP/processed_trajectories/md_dry_100ns_r3.dcd
!gdown --id 1OtEHAkJ-OhrPmtTor0Pfr3d_YVNvLIWX -O example-data/WT_298_2PET_RESP/processed_trajectories/md_dry_100ns_r4.dcd

/home/iwe30/anaconda3/envs/colabdesign/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=1SChRa8U5UPzLD9hTJLBhu-KouElG2w46
From (redirected): https://drive.google.com/uc?id=1SChRa8U5UPzLD9hTJLBhu-KouElG2w46&confirm=t&uuid=a879f6e4-1f8a-432f-a9b8-85d54d09089e
To: /home/iwe30/Github/md-analysis-utils/protein-ligand/example-data/WT_298_2PET_RESP/processed_trajectories/md_dry_100ns_r1.dcd
100%|████████████████████████████████████████| 938M/938M [00:19<00:00, 48.3MB/s]


## First calculate distances between matching AtomGroups from scratch and save them to a pickle file

In [18]:

systemsFolder = '/home/iwe30/Github/md-analysis-utils/protein-ligand/example-data'
# Select the OG atom of residue 130
serine_atom_list = ['OG']
serineResidueNumber = 130
pet_atoms_list = ['O9', 'O8', 'O7', 'O6', 'O1', 'O5', 'O2', 'O4', 'O3']
petResidueNumber = 259
 
figuresFolder = 'figures'
pickleFolder = 'pickles'
systemsDict = createSystemsDict(systemsFolder, '298')
systemName = 'WT_298_2PET_RESP'
universeDictByReplica = createUniverseDict(systemsDict, systemName)
# Run the loadAndPlotReplicaSystemFromPickle function for each system and replica 
# using the multiprocessing library starmap to run the function in parallel
# Open a pool of processes
from multiprocessing import Pool
import os
total_time = 50
# Set the number of processes to be the number of cores
numberOfProcesses = os.cpu_count()
print(f'Number of processes: {numberOfProcesses}')
# Get the number of replicas for the system by counting the max number in the pickles folder
# before the last underscore for the first system
numberOfReplicas = 4
# initialize the multiprocessing pool
pool = multiprocessing.Pool(processes=14)
# Create a list of arguments
# The first argument is the system name 
# The second argument is the replica number
# The third argument is the residue number of the serine
# The fourth argument is the residue number of the PET
# The fifth argument is the list of atoms in the serine
# The sixth argument is the list of atoms in the PET
# The seventh argument is the folder where the systems are located
# The eighth argument is the folder where the figures will be saved
args = [(key, replicaNumber, serineResidueNumber, petResidueNumber, serine_atom_list, pet_atoms_list, systemsFolder, figuresFolder) for key in systemsDict.keys() for replicaNumber in universeDictByReplica.keys()]
# Call the function calculateDistancesParallelFunction using the multiprocessing pool
pool.starmap(calculateDistancesParallelFunction, args)
# Close the multiprocessing pool
pool.close()


Creating universe for system: WT_298_2PET_RESP
Number of replicas:  4
Number of processes: 24
Creating universe for system: WT_298_2PET_RESPCreating universe for system: WT_298_2PET_RESPCreating universe for system: WT_298_2PET_RESPCreating universe for system: WT_298_2PET_RESP



Number of replicas: Number of replicas: Number of replicas: Number of replicas:    4 44
4




## Second create the plotly figures

In [26]:

# Desactivate future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

systemsDict = createSystemsDict(systemsFolder, '298')
# Run the loadAndPlotReplicaSystemFromPickle function for each system and replica 
# using the multiprocessing library starmap to run the function in parallel
# Open a pool of processes
from multiprocessing import Pool
import os
total_time = 100
# Set the number of processes to be the number of cores
numberOfProcesses = os.cpu_count()
print(f'Number of processes: {numberOfProcesses}')
# Get the number of replicas for the system by counting the max number in the pickles folder
# before the last underscore for the first system
numberOfReplicas = max([int(x.split('_')[-2]) for x in os.listdir(os.path.join(systemsFolder, pickleFolder)) if list(systemsDict.keys())[0] in x])
# Create a pool of processes
pool = Pool(numberOfProcesses)
# Run the loadAndPlotReplicaSystemFromPickle function for each system and replica
# using the multiprocessing library starmap to run the function in parallel
# First create the dfList
dfList = []
# Then create the list of tuples for the starmap function
args = [(total_time, 
         systemName, 
         dfList, 
         replicaNumber) 
            for systemName 
                in systemsDict 
                    for replicaNumber 
                        in 
                            range
                                (
                                    1, 
                                    numberOfReplicas + 1
                                )
            ]
# Run the starmap function
pool.starmap(loadAndPlotReplicaSystemFromPickle, args)
# Close the pool of processes
pool.close()






Number of processes: 24
/home/iwe30/Github/md-analysis-utils/protein-ligand/example-data/figures/WT_298_2PET_RESP_1_distances.png has been saved
/home/iwe30/Github/md-analysis-utils/protein-ligand/example-data/figures/WT_298_2PET_RESP_4_distances.png has been saved
/home/iwe30/Github/md-analysis-utils/protein-ligand/example-data/figures/WT_298_2PET_RESP_3_distances.png has been saved
/home/iwe30/Github/md-analysis-utils/protein-ligand/example-data/figures/WT_298_2PET_RESP_2_distances.png has been saved
